# Arbitrary Neural Style Transfer using Pretrained Models

Google's Magenta project provides pre-trained models for real-time style transfer using arbitrary images and styles. 
[Check out the full repository here.](https://github.com/magenta/magenta/tree/master/magenta/models/arbitrary_image_stylization)

In addition to the model, I set up a specific directory structure to match my images. This folder is called `Selfies` and is at the root of my google drive. The structure is as follows:
```
Selfies/
    output/
      # Output files will be stored here
    styles/
      style-0.jpg
      style-1.jpg
      style-2.jpg
      style-3.jpg
      style-4.jpg
      style-5.jpg
      style-6.jpg
      style-7.jpg
    videos/
      video-1.mp4
    pictures/
      img-1.jpg
      img-2.jpg
      ...
```

The photos in the `pictures` directory can have any number of images and any filename as long as the extension is `.jpg`. 

The code below evaluates 8 styles on the video `video-1.mp4`. 

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! wget https://storage.googleapis.com/download.magenta.tensorflow.org/models/arbitrary_style_transfer.tar.gz

--2021-05-16 07:05:13--  https://storage.googleapis.com/download.magenta.tensorflow.org/models/arbitrary_style_transfer.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.136.128, 142.250.148.128, 209.85.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.136.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 635651017 (606M) [application/gzip]
Saving to: ‘arbitrary_style_transfer.tar.gz’

arbitrary_style_tra 100%[===================>] 606.20M  82.4MB/s    in 6.8s    

2021-05-16 07:05:20 (89.5 MB/s) - ‘arbitrary_style_transfer.tar.gz’ saved [635651017/635651017]



In [ ]:
! tar -xzvf arbitrary_style_transfer.tar.gz

arbitrary_style_transfer/
arbitrary_style_transfer/model.ckpt.index
arbitrary_style_transfer/model.ckpt.data-00000-of-00001
arbitrary_style_transfer/model.ckpt.meta


In [ ]:
! git clone https://github.com/tensorflow/magenta.git

Cloning into 'magenta'...
remote: Enumerating objects: 15165, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 15165 (delta 13), reused 19 (delta 10), pack-reused 15128
Receiving objects: 100% (15165/15165), 36.12 MiB | 31.29 MiB/s, done.
Resolving deltas: 100% (11443/11443), done.


In [ ]:
! pip install -e magenta

Obtaining file:///content/magenta
     |████████████████████████████████| 256kB 6.6MB/s 
     |████████████████████████████████| 1.6MB 9.1MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
     |████████████████████████████████| 92kB 7.0MB/s 
     |████████████████████████████████| 215kB 18.7MB/s 
     |████████████████████████████████| 3.6MB 25.2MB/s 
     |████████████████████████████████| 5.6MB 20.5MB/s 
     |████████████████████████████████| 204kB 52.5MB/s 
     |████████████████████████████████| 2.3MB 50.7MB/s 
     |████████████████████████████████| 1.5MB 48.2MB/s 
     |████████████████████████████████| 358kB 53.3MB/s 
     |████████████████████████████████| 20.2MB 1.5MB/s 
     |████████████████████████████████| 655kB 47.6MB/s 
     |████████████████████████████████| 368kB 49.0MB/s 
     |████████████████████████████████| 194kB 49.5MB/s 
     |████████████████████████████████| 5.6MB 49.0MB/s 
     |████████████████████████████████| 81kB 7.7MB/s 
     |████████████████

In [ ]:
import os
import sys
import cv2
import time

def stylize_frames(style_path, output_folder, image_folder_base):
  os.makedirs(output_folder, exist_ok=True)
  os.system("! python magenta/magenta/models/arbitrary_image_stylization/arbitrary_image_stylization_with_weights.py \
          --checkpoint=arbitrary_style_transfer/model.ckpt \
          --output_dir={} \
          --style_images_paths={} \
          --content_images_paths={}/*\
          --image_size=256 \
          --content_square_crop=True \
          --style_image_size=256 \
          --style_square_crop=True".format(output_folder, style_path, image_folder_base))

def video_name_stripped(video_path):
  return video_path.split('.')[0].split('/')[-1]

def extract_video_frames(video_filename, output_path):
  folder_base = os.path.join(output_path, video_name_stripped(video_filename))
  os.makedirs(folder_base, exist_ok=True)
  vidcap = cv2.VideoCapture(video_filename)
  success, image = vidcap.read()
  count = 0
  while success:
    cv2.imwrite(os.path.join(folder_base, "frame_%06d.jpg" % count), image)     # save frame as JPEG file      
    success,image = vidcap.read()
    if count % 1000 == 0: 
      print('Read a new frame: ', success, count)
    count += 1

  return folder_base

def style_video_pipeline(style_path, image_folder_base, output_path):
  print("Stylizing Individual Frames...")
  stylized_folder = os.path.join(output_path, video_name_stripped(style_path))
  os.makedirs(stylized_folder, exist_ok=True)
  %time stylize_frames(style_path, stylized_folder, image_folder_base)
  print("Stitching Styled video...")
  %time os.system("ffmpeg -hide_banner -loglevel error -framerate 60 -pattern_type glob -i '{}/*stylized*.jpg'  -c:v libx264 -r 60 -pix_fmt yuv420p {}/{}.mp4".format(stylized_folder, output_path, video_name_stripped(style_path)))

In [ ]:
video_output_path = "/content/drive/MyDrive/Selfies/videos"
video_path = "/content/drive/MyDrive/Selfies/videos/video_1.mp4"
%time image_folder_base = extract_video_frames(video_path, video_output_path)

Read a new frame:  True 0
Read a new frame:  True 1000
CPU times: user 1min 43s, sys: 5.14 s, total: 1min 49s
Wall time: 1min 46s


In [ ]:
output_path = "/content/drive/MyDrive/Selfies/output"
style_path_base = "/content/drive/MyDrive/Selfies/styles/style-{}.jpeg"
selfie_folder_base = "/content/drive/MyDrive/Selfies/pictures"
for i in range(8):
  print("Styling a few selfies first...")
  %time stylize_frames(style_path_base.format(i), os.path.join(output_path, 'selfies_{}'.format(i)), selfie_folder_base)
  print("Styling a 30 second video at 60 fps...")
  style_video_pipeline(style_path_base.format(i), image_folder_base, output_path)

Styling a few selfies first...
CPU times: user 128 ms, sys: 25.6 ms, total: 154 ms
Wall time: 30 s
Styling a 30 second video at 60 fps...
Stylizing Individual Frames...
CPU times: user 1.97 s, sys: 240 ms, total: 2.21 s
Wall time: 9min 34s
Stitching Styled video...
CPU times: user 11.7 ms, sys: 15.5 ms, total: 27.1 ms
Wall time: 5.43 s
Styling a few selfies first...
CPU times: user 44.2 ms, sys: 8.56 ms, total: 52.8 ms
Wall time: 18.8 s
Styling a 30 second video at 60 fps...
Stylizing Individual Frames...
CPU times: user 1.27 s, sys: 125 ms, total: 1.4 s
Wall time: 9min 38s
Stitching Styled video...
CPU times: user 11.8 ms, sys: 14.1 ms, total: 25.9 ms
Wall time: 4.42 s
Styling a few selfies first...
CPU times: user 44 ms, sys: 9.44 ms, total: 53.4 ms
Wall time: 19.6 s
Styling a 30 second video at 60 fps...
Stylizing Individual Frames...
CPU times: user 1.44 s, sys: 118 ms, total: 1.55 s
Wall time: 10min 20s
Stitching Styled video...
CPU times: user 12.1 ms, sys: 13.3 ms, total: 25.4 m